410921202 資工三 林芷萱 HW3

In [8]:
import cv2
import numpy as np
import mediapipe as mp

mustache = cv2.imread('mustache.png', cv2.IMREAD_UNCHANGED)
hat = cv2.imread('hat.png', cv2.IMREAD_UNCHANGED)

mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils

cap = cv2.VideoCapture(0)

# overlay_image_alpha用來將img_overlay疊加到img上面
def overlay_image_alpha(img, img_overlay, pos, alpha_mask):
    x, y = pos

    # 原始圖片的大小
    y1, y2 = max(0, y), min(img.shape[0], y + img_overlay.shape[0])
    x1, x2 = max(0, x), min(img.shape[1], x + img_overlay.shape[1])
    # 要疊加的圖片大小
    y1o, y2o = max(0, -y), min(img_overlay.shape[0], img.shape[0] - y)
    x1o, x2o = max(0, -x), min(img_overlay.shape[1], img.shape[1] - x)

    # 檢查是否重疊，如果沒重疊->範圍無效，return
    if y1 >= y2 or x1 >= x2 or y1o >= y2o or x1o >= x2o:
        return

    # 疊加圖片
    img_crop = img[y1:y2, x1:x2] # 裁減的原始圖片
    img_overlay_crop = img_overlay[y1o:y2o, x1o:x2o] # 裁減的疊加圖片
    alpha = alpha_mask[y1o:y2o, x1o:x2o, np.newaxis]
    
    np.multiply(alpha, img_overlay_crop, out=img_overlay_crop, casting="unsafe") # 把alpha跟疊加圖片相乘
    np.multiply(1.0 - alpha, img_crop, out=img_crop, casting="unsafe") # 把1-alpha跟原始圖片相乘
    np.add(img_crop, img_overlay_crop, out=img_crop, casting="unsafe") # 處理過後的圖片相加

with mp_face_detection.FaceDetection(min_detection_confidence=0.5) as face_detection:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Flip the image horizontally for a later selfie-view display, and convert
        # the BGR image to RGB.
        frame = cv2.flip(frame, 1)
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Run MediaPipe Face Detection.
        results = face_detection.process(rgb_frame)
        
        if results.detections:
            for detection in results.detections:
                # 取得臉部的bounding box座標
                bboxC = detection.location_data.relative_bounding_box # 臉部的bounding box
                ih, iw, _ = frame.shape # 圖片長&寬
                # bounding box的座標跟長寬
                x1, y1, w, h = int(bboxC.xmin * iw), int(bboxC.ymin * ih), int(bboxC.width * iw), int(bboxC.height * ih)
                                
                # 取得keypoints跟座標
                keypoints = detection.location_data.relative_keypoints
                right_eye = (int(keypoints[0].x * iw), int(keypoints[0].y * ih))
                left_eye = (int(keypoints[1].x * iw), int(keypoints[1].y * ih))
                nose_tip = (int(keypoints[2].x * iw), int(keypoints[2].y * ih))
                mouth_center = (int(keypoints[3].x * iw), int(keypoints[3].y * ih))

                # 計算疊加上去的圖像的位置跟尺寸
                # 鬍子
                mustache_width = w // 2
                mustache_height = int(mustache.shape[0] * mustache_width / mustache.shape[1])
                mustache_pos = (nose_tip[0] - mustache_width // 2, nose_tip[1] + 10) # 根據鼻尖在哪裡放鬍子
                # 帽子
                hat_width = w
                hat_height = int(hat.shape[0] * hat_width / hat.shape[1])
                hat_pos = (x1, y1 - hat_height + 20) # 帽子放在頭的上面
                
                # 計算疊加上去的圖像大小
                mustache_resized = cv2.resize(mustache, (mustache_width, mustache_height), interpolation=cv2.INTER_AREA)
                hat_resized = cv2.resize(hat, (hat_width, hat_height), interpolation=cv2.INTER_AREA)
                
                # 取得疊加用的Alpha通道
                # 鬍子
                mustache_alpha = mustache_resized[:, :, 3] / 255.0
                mustache_resized = mustache_resized[:, :, :3]
                # 帽子
                hat_alpha = hat_resized[:, :, 3] / 255.0
                hat_resized = hat_resized[:, :, :3]
                
                # 疊加鬍子跟帽子
                overlay_image_alpha(frame, mustache_resized, mustache_pos, mustache_alpha)
                overlay_image_alpha(frame, hat_resized, hat_pos, hat_alpha)
        
        cv2.imshow('output', frame)

        c = cv2.waitKey(1)
        if c == 27:
            break

cap.release()
cv2.destroyAllWindows()